<a href="https://colab.research.google.com/drive/1GHS6wHyaJ-etayVmL-r6eqTjYUjuWE-j?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### импорт библиотек

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
from datetime import datetime
from tqdm.notebook import tqdm

import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
from scipy.stats import uniform
import torch.nn.functional as F

import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torch.nn as nn
import math
import torch.optim as optim

import re
import time
import nltk
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tqdm import tqdm
from gensim import models
import gensim
from gensim.models import Word2Vec
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import TfidfModel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import *

from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_validate
from sklearn import metrics
from sklearn.preprocessing import MaxAbsScaler, StandardScaler, MinMaxScaler
from sklearn.preprocessing import normalize


import warnings
warnings.filterwarnings("ignore")

# from tqdm import tqdm, tqdm_notebook
from tqdm import tqdm
tqdm.pandas()

### загрузка данных

In [ ]:
train = pd.read_pickle("/content/train.df", compression="gzip") # ссылка на датафрейм https://www.kaggle.com/datasets/luydmilateplova/lenta-f
train

In [ ]:
train = pd.read_pickle("/content/train.df", compression="gzip")
test_data = pd.read_pickle("/content/test.df", compression="gzip")

In [ ]:
!unzip test_news.csv.zip

In [ ]:
# добавление кусочков (по 2000 строк) датафреймов в единый датасет

f1 = pd.read_pickle("/content/fontanka_df2023.df", compression="gzip")
f2 = pd.read_pickle("/content/fontanka_df2022.df", compression="gzip")
f3 = pd.read_pickle("/content/fontanka_df2021.df", compression="gzip")
f4 = pd.read_pickle("/content/fontanka_df2020.df", compression="gzip")
f5 = pd.read_pickle("/content/fontanka_df2019.df", compression="gzip")


# f2 = pd.read_pickle("/content/fontanka_df21.df", compression="gzip")
# f3 = pd.read_pickle("/content/fontanka_df22_23.df", compression="gzip")

fontan =  pd.concat((f1, f2, f3, f4, f5), axis=0)
fontan


In [ ]:
fontan[fontan.duplicated()]

In [ ]:
# удаление дубликатов
data_fontan = fontan.drop_duplicates(keep= False )
data_fontan

In [ ]:
# читаем тестовый датасет
test_data = pd.read_csv('/content/test_news.csv')
test_data

In [ ]:
# перемешивание данных в датафрейме
# from sklearn.utils import shuffle
# train_df = shuffle(train_df)
# train_df

In [ ]:
# склеивание кусочков спарсенных датафреймов
# dff = ['01', '02', '03', '04', '05', '06', '07', '05', '06', '07', '08', '09'] + list(range(10, 76))
# # ['01', '02', '03', '04', '05', '06', '07', '05', '06', '07', '08', '09']  + list(range(10, 76))
# data = pd.DataFrame()
# for i in dff:
#     dat = pd.read_pickle(f"{i}.df", compression="gzip")
#     data =  pd.concat((data, dat), axis=0)


In [ ]:
data = pd.read_pickle('/content/lenta_all.df', compression="gzip")
data

In [ ]:
set(data.category)

In [ ]:
# смотрим распределение категорий
def loads_labels(dataset):
    labels = []
    for i in dataset:
        labels.append(i)
    counts = Counter(labels)
    count_df =  pd.DataFrame([(k, v) for k, v in counts.items()],
    columns=['category', 'count'])
    return count_df.sort_values(by = 'count', ascending = False)

In [ ]:
df = loads_labels(train.target.values)
df.reset_index()

In [ ]:
df['target'] = df['category'].map({0: 'Россия', 1: 'Экономика', 2 : 'Силовые структуры',
                                   3 : 'Бывший СССР', 4 : 'Спорт', 5 : 'Забота о себе', 6 : 'Строительство', 7 : 'Путешествия', 8 : 'Наука и техника'})

In [ ]:
df

In [ ]:
ax = df.plot.barh(x='target', y='count', rot=0, figsize = (10,4), title='Распределение - категория / количество',
                     xlabel='Количество / категория',
                     ylabel='категория')

In [ ]:
df = loads_labels(data_t.category.values)
df.reset_index()

### препроцессинг

In [ ]:
# кодировка категорий
data['target'] = data['category'].map({'Россия': 0, 'Экономика' : 1, 'Силовые структуры' : 2, 'Бывший СССР': 3, 'Спорт' : 4, 'Забота о себе' : 5, 'Путешествия' : 7, 'Наука и техника' : 8})

In [ ]:
data

In [ ]:
# ! pip install pymorphy2
# import pymorphy2
# m = pymorphy2.MorphAnalyzer()

In [ ]:
# препроцессинг текста

# mystopwords = stopwords.words('russian') + ['фото', 'фонтанка', 'риа', 'ру',
#     'это', 'наш' , 'тыс', 'млн', 'млрд', 'также',  'т', 'д',
#     'который','прошлый','сей', 'свой', 'наш', 'мочь', 'такой'
# ]

ru_words = re.compile("[а-яА-Я0-9]+")

# lemmatizer = nltk.WordNetLemmatizer()

def words_only(text):
    return " ".join(ru_words.findall(text))

# лемматизация
# def lemmatize(text):
#     try:
#         # return  " ".join([m.parse(w)[0].normal_form for w in text.lower().split()])
#         return " ".join([lemmatizer.lemmatize(w) for w in text.lower().split()])
#     except:
#         return " "

# удаление стоп-слов и всех пунктац. знаков
def remove_stopwords(text):
    try:
        return " ".join([token for token in text.split()])
    except:
        return ""


def preprocess(text):
    return remove_stopwords(words_only(text.lower()))

In [ ]:
data['text_prep'] = data.text.astype('str').progress_apply(preprocess)

In [ ]:
data

In [ ]:
# формирование датафрейма, который будем подавать модели
train = pd.DataFrame({'link' : data.link, 'text' : data.text, 'target' : data.target})
train

In [ ]:
# соединение датафреймов с ленты и фонтанки
train = pd.concat((train, data_fontan), axis = 0 )
train

In [ ]:
# перемешивание данных
from sklearn.utils import shuffle
train = shuffle(train)

In [ ]:
train

### обучение sklearn

In [ ]:
# готовим столбец, который будем подавать в модель (text_prep)
train['text_prep'] = train.text.astype('str').progress_apply(preprocess)
train

In [ ]:
# делим на выборки для обучения
y = train.target
X_train, X_val, y_train, y_val = train_test_split(train.text_prep, y, test_size=0.2, random_state=42)

# tfidf = TfidfVectorizer(min_df=6, max_df=1.0, ngram_range=(1, 1))
# tfidf = TfidfVectorizer(min_df=3, max_df=0.5, ngram_range=(1, 1))
tfidf = TfidfVectorizer(min_df=5, max_df=0.2, ngram_range=(1, 2))

X_train = tfidf.fit_transform(X_train)
X_val = tfidf.transform(X_val)

# Нормализация матрицы TF-IDF по длине документа
X_train = normalize(X_train, norm='l2', axis=1)
X_val = normalize(X_val, norm='l2', axis=1)

In [ ]:
print ((X_train.shape),(y_train.shape),(X_val.shape),(y_val.shape))

In [ ]:
# sc = StandardScaler()
# X_tr = sc.fit_transform(X_train)
# X_te = sc.transform(X_test)

In [ ]:
models = [
    RandomForestClassifier(n_estimators=100, random_state = 42),
    LogisticRegression(C = 10, max_iter = 100, class_weight='balanced', random_state = 42),
    SGDClassifier(alpha=0.001, loss='log_loss', tol = 0.0004, epsilon=0.01, max_iter=10, class_weight='balanced', learning_rate = 'optimal', shuffle = False, random_state=42),
    SVC(kernel = 'linear', gamma = 'scale', class_weight='balanced', random_state = 42),
    XGBClassifier(n_estimators = 200, max_depth = 2, nthread = 8, learning_rate = 0.2)
    ]

In [ ]:
# функция для обучения нескольких моделей с сохранением результата в датафрейм
def train_regress(models):

    score = {}
    for model in models:
        score_a_p = []

        model.fit(X_train, y_train)
        pred = model.predict(X_val)
        predictions = [round(el) for el in pred]
        accuracy = accuracy_score(y_val, predictions)
        precision = precision_score(y_val, pred, average='weighted')
        name = re.findall(r'\w+\(', str(model))
        name = str(name[0].split('(')[0])
        score_a_p.append(accuracy)
        score_a_p.append(precision)
        score[name] = score_a_p

    result = pd.DataFrame.from_dict(score, orient='index').reset_index()
    result['model'] = result['index']
    result['accuracy'] = result[0]
    result['precision'] = result[1]
    result = result.drop(columns=[0, 1, 'index'])
    result = result.sort_values(by = ['accuracy'], ascending = False)
    result

    return result

In [ ]:
df_res = train_regress(models)

In [ ]:
df_res

In [ ]:
# пайплан бэйзлайна

# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.preprocessing import MaxAbsScaler
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import classification_report

# y = train.target
# X_train, X_val, y_train, y_val = train_test_split(train.text_prep, y, test_size=0.2, random_state=42)

# # vec = CountVectorizer()
# vec = CountVectorizer(max_df=0.8, min_df=1.0, max_features = 5000)

# vec.fit(X_train)

# bow = vec.transform(X_train)  # bow — bag of words (мешок слов)
# bow_test = vec.transform(X_val)

# print(bow.shape)

# scaler = MaxAbsScaler()
# bow = scaler.fit_transform(bow)
# bow_test = scaler.transform(bow_test)

# clf = LogisticRegression(max_iter=200, random_state=42)
# clf.fit(bow, y_train)
# pred = clf.predict(bow_test)

# print(classification_report(y_val, pred))

In [ ]:
lr_clf = LogisticRegression(C = 10, max_iter = 400, class_weight='balanced',  random_state = 42)
lr_clf.fit(X_train, y_train)

lr_train_pred = lr_clf.predict(X_train)
lr_val_pred = lr_clf.predict(X_val)

accuracy_score(y_train, lr_train_pred), accuracy_score(y_val, lr_val_pred)

In [ ]:
# sgd_clf = SGDClassifier(loss='log_loss', max_iter=3000, epsilon=0.01, class_weight='balanced', random_state=42)
sgd_clf = SGDClassifier(loss = 'modified_huber', max_iter=5000, class_weight='balanced', random_state=42)

# epsilon=0.01, max_iter=3000

sgd_clf.fit(X_train, y_train)

sgd_clf_train_pred = sgd_clf.predict(X_train)
sgd_clf_val_pred = sgd_clf.predict(X_val)

accuracy_score(y_train, sgd_clf_train_pred), accuracy_score(y_val, sgd_clf_val_pred)

In [ ]:
xgb = XGBClassifier(n_estimators = 200, max_depth = 2, nthread = 8, learning_rate = 0.2)
xgb.fit(X_train, y_train)

xgb_train_pred = xgb.predict(X_train)
xgb_val_pred = xgb.predict(X_val)

accuracy_score(y_train, xgb_train_pred), accuracy_score(y_val, xgb_val_pred)


In [ ]:
rf_clf = RandomForestClassifier(n_estimators=500,  random_state = 42)

rf_clf.fit(X_train, y_train)

rf_train_pred = rf_clf.predict(X_train)
rf_val_pred = rf_clf.predict(X_val)

accuracy_score(y_train, rf_train_pred), accuracy_score(y_val, rf_val_pred)


### stacking

In [ ]:
models = [
    RandomForestClassifier(n_estimators=100, random_state = 42),
    LogisticRegression(C = 10, max_iter = 100, class_weight='balanced', random_state = 42),
    # DummyClassifier(strategy="prior", random_state = 42),
    SGDClassifier(alpha=0.001, loss='log_loss', tol = 0.0004, epsilon=0.01, max_iter=10, class_weight='balanced', learning_rate = 'optimal', shuffle = False, random_state=42),
    SVC(kernel = 'linear', gamma = 'scale', class_weight='balanced', random_state = 42),
    XGBClassifier(n_estimators = 200, max_depth = 2, nthread = 8, learning_rate = 0.2)
    ]

In [ ]:
# model_svc = SVC(kernel = 'linear', gamma = 'scale')
# model_rfc = RandomForestClassifier(criterion = 'gini', n_estimators = 100, min_samples_split = 7, min_samples_leaf= 2, max_features = 'sqrt', max_depth= 5, bootstrap=True, random_state=42)
# model_logr = LogisticRegression(penalty = 'l2', C = 10, multi_class = 'multinomial', max_iter = 5000, solver = 'newton-cg')
# # model_gbr = GradientBoostingRegressor(n_estimators = 100, max_depth = 10 , learning_rate = 0.04, min_samples_leaf = 15, random_state=42)
# model_ridge = Ridge(alpha = 0.43206891751952314)
# # model_ridge = Ridge()

# # model.fit(X_train, y_train)

In [ ]:
# Создаем стэккинг и обучаем его на наших данных
stacking = StackingClassifier(
    [   ('LogisticRegression', models[1]),
        ('RandomForestClassifier', models[0]),
        ('SGD', models[2]),
        ('SVC', models[3]),
        ('XGB', models[4]),
    ])

In [ ]:
model_st = stacking.fit(X_train, y_train)
# model = stacking.fit(X_poly,y_train)


In [ ]:
train_accuracy_score=model_st.score(X_train,y_train)
print(train_accuracy_score)

test_accuracy_score=model_st.score(X_val,y_val)
print(test_accuracy_score)

In [ ]:
# ridge = RidgeCV(cv=3)
# ridge.fit(X_train, y_train)
# ridge_train_pred = ridge.predict(X_train)
# predictions_train = [round(el) for el in ridge_train_pred]

# ridge_val_pred = ridge.predict(X_val)
# predictions_val = [round(el) for el in ridge_val_pred]

# accuracy_score(y_train, predictions_train), accuracy_score(y_val, predictions_val)

### Word2Vec

In [ ]:
# токенизация
repliks = []
for el in train.text_prep:
    sentences = nltk.sent_tokenize(el, language="russian")
    for sentence in sentences:
        tokens = nltk.word_tokenize(sentence)
        repliks.append(tokens)

In [ ]:
%time  model_w2v = Word2Vec(repliks, workers=4, vector_size=300, min_count=5, window=10)
model_w2v.build_vocab(repliks) # построение словаря
model_w2v.train(repliks, total_examples=model_w2v.corpus_count, epochs=model_w2v.epochs)

In [ ]:
vocab_len = len(model_w2v.wv) # длина словаря
vocab_len

In [ ]:
# вывод близких слов
model_w2v.wv.most_similar(positive=["ехать"], topn=5)

In [ ]:
# создание эмбеддингов
def get_text_embedding(text):
    result = []
    for word in word_tokenize(text.lower()):
        if word in model_w2v.wv:
            result.append(model_w2v.wv[word])

    if len(result):
        result = np.average(result, axis=0)
    else:
        result = np.zeros(300)
    return result


In [ ]:
features = [get_text_embedding(text) for text in tqdm(train.text_prep.values)]


In [ ]:
len(features)

In [ ]:
len(features[0])

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(features, train.target.values, test_size=0.20, random_state=42)
print (' train:', len(X_train), len(y_train), '\n', 'test:', len(X_val),len(y_val))

In [ ]:
# sc = MinMaxScaler()
# X_train = sc.fit_transform(X_train)
# X_test = sc.transform(X_val)

In [ ]:
df_res_w2v = train_regress(models)

In [ ]:
df_res_w2v

In [ ]:
sgd_clf = SGDClassifier(loss='log_loss', max_iter=3000, epsilon=0.01, class_weight='balanced', random_state=42)
# sgd_clf = SGDClassifier(loss = 'modified_huber', max_iter=3000, epsilon=0.01, class_weight='balanced', random_state=42)
# sgd_clf = SGDClassifier(max_iter=2000, epsilon=0.01, class_weight='balanced', random_state=42)


# epsilon=0.01, max_iter=3000

sgd_clf.fit(X_train, y_train)

sgd_clf_train_pred = sgd_clf.predict(X_train)
sgd_clf_val_pred = sgd_clf.predict(X_val)

accuracy_score(y_train, sgd_clf_train_pred), accuracy_score(y_val, sgd_clf_val_pred)

In [ ]:
lr_clf = LogisticRegression(max_iter = 1000, class_weight='balanced', random_state = 42)
lr_clf.fit(X_train, y_train)

lr_train_pred = lr_clf.predict(X_train)
lr_val_pred = lr_clf.predict(X_val)

accuracy_score(y_train, lr_train_pred), accuracy_score(y_val, lr_val_pred)


# svc_clf = SVC(kernel = 'linear', gamma = 'scale', class_weight='balanced', random_state = 42)
# svc_clf.fit(X_train, y_train)

# svc_train_pred = svc_clf.predict(X_train)
# svc_val_pred = svc_clf.predict(X_val)

# accuracy_score(y_train, svc_train_pred), accuracy_score(y_val, svc_val_pred)

# (0.9446800182660317, 0.9123402034959561)


In [ ]:
svc_clf = SVC(kernel = 'linear', gamma = 'scale', class_weight='balanced', random_state = 42)
svc_clf.fit(X_train, y_train)

svc_train_pred = svc_clf.predict(X_train)
svc_val_pred = svc_clf.predict(X_val)

accuracy_score(y_train, svc_train_pred), accuracy_score(y_val, svc_val_pred)

# (0.9446800182660317, 0.9123402034959561)

In [ ]:
svc_clf = RandomForestClassifier(n_estimators=100, random_state = 42)

svc_clf.fit(X_train, y_train)

svc_train_pred = svc_clf.predict(X_train)
svc_val_pred = svc_clf.predict(X_val)

accuracy_score(y_train, svc_train_pred), accuracy_score(y_val, svc_val_pred)


In [ ]:
svc_clf.fit(features, y)

In [ ]:
svc_train_pred = svc_clf.predict(features)
accuracy_score(y, svc_train_pred)


In [ ]:
svc_train_pred

### bert

In [ ]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")
device

In [ ]:
!pip install transformers

In [ ]:
sentences = train_df.text_prep.values
labels = train_df.category.values

In [ ]:
# токенезация
from transformers import BertTokenizer,AutoTokenizer, AutoModelForMaskedLM,AutoModelForSequenceClassification
from transformers import DebertaTokenizer
from transformers import RobertaTokenizer, RobertaForSequenceClassification
print('Loading BERT tokenizer...')
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2", do_lower_case=True)
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased", do_lower_case=True)


# model = AutoModel.from_pretrained("cointegrated/rubert-tiny2")

# tokenizer = AutoTokenizer.from_pretrained('distilroberta-base', do_lower_case=True)
# tokenizer = AutoTokenizer.from_pretrained('roberta-large', do_lower_case=True)

In [ ]:
print(' Original: ', sentences[0])
print('Tokenized: ', tokenizer.tokenize(sentences[0]))
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0])))

In [ ]:
input_ids = []
attention_masks = []

for sent in sentences:
    encoded_dict = tokenizer.encode_plus(
                        sent,
                        add_special_tokens = True,
                        max_length = 256,
                        truncation = True,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt',
                   )

    input_ids.append(encoded_dict['input_ids'])

    attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

In [ ]:
from torch.utils.data import TensorDataset, random_split

dataset = TensorDataset(input_ids, attention_masks, labels)

train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

In [ ]:
labels.shape

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 16

train_dataloader = DataLoader(
            train_dataset,
            sampler = RandomSampler(train_dataset),
            batch_size = batch_size
        )


validation_dataloader = DataLoader(
            val_dataset,
            sampler = SequentialSampler(val_dataset),
            batch_size = batch_size
        )

In [ ]:
from transformers import BertForSequenceClassification, AdamW, DebertaForSequenceClassification

# model = AutoModel.from_pretrained("cointegrated/rubert-tiny2")

model = BertForSequenceClassification.from_pretrained(
    # "bert-base-uncased",
    # "cointegrated/rubert-tiny2",
    'DeepPavlov/rubert-base-cased',
    num_labels = 9,
    output_attentions = False,
    output_hidden_states = False,
)

model.cuda()

In [ ]:
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5,
                  eps = 1e-8
                )


In [ ]:
from transformers import get_linear_schedule_with_warmup

epochs = 4

total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [ ]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
import time
import datetime

def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
import random

seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

training_stats = []

total_t0 = time.time()

for epoch_i in range(0, epochs):

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()
    total_train_loss = 0
    model.train()

    for step, batch in enumerate(train_dataloader):
        if step % 40 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        model.zero_grad()

        res = model(b_input_ids,
                             token_type_ids=None,
                             attention_mask=b_input_mask,
                             labels=b_labels)
        loss = res['loss']
        logits = res['logits']
        total_train_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
    avg_train_loss = total_train_loss / len(train_dataloader)
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
    print("")
    print("Running Validation...")

    t0 = time.time()
    model.eval()

    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    for batch in validation_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        with torch.no_grad():
            res = model(b_input_ids,
                                   token_type_ids=None,
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
        loss = res['loss']
        logits = res['logits']

        total_eval_loss += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        total_eval_accuracy += flat_accuracy(logits, label_ids)

    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    avg_val_loss = total_eval_loss / len(validation_dataloader)

    validation_time = format_time(time.time() - t0)

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

In [ ]:
pd.options.display.precision
df_stats = pd.DataFrame(data=training_stats)
df_stats = df_stats.set_index('epoch')
df_stats

In [ ]:
%matplotlib inline
import seaborn as sns

sns.set(style='darkgrid')
sns.set(font_scale=1.5)
plt.rcParams["figure.figsize"] = (10,6)
plt.plot(df_stats['Training Loss'], 'b-o', label="Loss")
plt.plot(df_stats['Valid. Loss'], 'g-o', label="Accuracy")
plt.title("BERT валидационные метрики")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.xticks([1, 2, 3, 4])

plt.show()

In [ ]:
torch.save(model.state_dict(), "bert_4epoch.pth")

In [ ]:
df = test_df
df

In [ ]:
import pandas as pd

print('Number of test sentences: {:,}\n'.format(df.shape[0]))

sentences = df.text_prep.values

input_ids = []
attention_masks = []

for sent in sentences:
    encoded_dict = tokenizer.encode_plus(
                        sent,
                        add_special_tokens = True,
                        max_length = 300,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt',
                   )

    input_ids.append(encoded_dict['input_ids'])

    attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)

batch_size = 64

prediction_data = TensorDataset(input_ids, attention_masks)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

In [ ]:
print('Predicting labels for {:,} test sentences...'.format(len(input_ids)))

model.eval()

predictions , true_labels = [], []
for batch in prediction_dataloader:
  batch = tuple(t.to(device) for t in batch)
  b_input_ids, b_input_mask = batch
  with torch.no_grad():
      outputs = model(b_input_ids, token_type_ids=None,
                      attention_mask=b_input_mask)

  logits = outputs[0]
  logits = logits.detach().cpu().numpy()
#   label_ids = b_labels.to('cpu').numpy()

  predictions.append(logits)
print('    DONE.')

In [ ]:
flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()

In [ ]:
flat_predictions

In [ ]:
# df["groups"] = flat_predictions
# df = df.drop("content", axis=1)
# df

### обработка тестового датасета

In [ ]:
test_data = pd.read_csv('/content/test_news.csv')


In [ ]:
# test_data['text_prep'] = test_data.content.astype('str').progress_apply(preprocess)
test_data

In [ ]:
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.preprocessing import MaxAbsScaler
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import classification_report

# vec = CountVectorizer(max_df=0.8, min_df=2, max_features = 1000)
# vec.fit(train.text_prep)

# bow = vec.transform(train.text_prep)  # bow — bag of words (мешок слов)
# bow_test = vec.transform(test_data.text_prep)

# scaler = MaxAbsScaler()
# bow = scaler.fit_transform(bow)
# bow_test = scaler.transform(bow_test)

# clf = LogisticRegression(max_iter=200, random_state=42)
# clf.fit(bow, y)
# pred = clf.predict(bow_test)

In [ ]:
# создание эмбеддингов
# def get_text_embedding(text):
#     result = []
#     for word in word_tokenize(text.lower()):
#         if word in model_w2v.wv:
#             result.append(model_w2v.wv[word])

#     if len(result):
#         result = np.average(result, axis=0)
#     else:
#         result = np.zeros(300)
#     return result

features_t = [get_text_embedding(text) for text in tqdm(test_data.text_prep.values)]


In [ ]:
X = features
X_test = features_t

lr_clf = lr_clf

lr_clf.fit(X, y)
lr_test_pred = lr_clf.predict(X_test)
lr_test_pred

In [ ]:
# Решение, применяем пайплайн для train и test

X = tfidf.fit_transform(train.text_prep) # берем весь train без разбивки
X_test = tfidf.transform(test_data.text_prep) # берем тест, на котором строим сабмит

# sc = MaxAbsScaler()
# X = sc.fit_transform(X)
# X_test = sc.transform(X_test)

# # Нормализация матрицы TF-IDF по длине документа
X = normalize(X, norm='l2', axis=1)
X_test = normalize(X_test, norm='l2', axis=1)

# обучение (можно менять модели)
lr_clf = sgd_clf
lr_clf.fit(X, y)

# предсказание на тесте
lr_test_pred = lr_clf.predict(X_test)
lr_test_pred

### сабмит

In [ ]:
result = pd.DataFrame({'topic': lr_test_pred, 'index' : test_data.index.values})

result

In [ ]:
result.to_csv(r'submission_news.csv', index= False )

In [ ]:
# # train.to_pickle("train.df", compression="gzip")
# test_data.to_pickle("test.df", compression="gzip")